# Segmenting and Clustering Neighborhoods in Toronto

### Import important libraries

In [1]:
from bs4 import BeautifulSoup
import requests   # library to handle requests
import lxml       # parse the website in lxml format
import numpy as np
import pandas as pd

### Scraping website

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table', class_='wikitable sortable')
# print(table.prettify())

### Preprocessing the table and getting important values

In [3]:
# headers="Postcode,Borough,Neighbourhood"
table1 = ""
for tr in table.find_all('tr'):
    row = ""
    for tds in tr.find_all('td'):
        row = row + "," + tds.text
    table1 = table1 + row[1:]
# print(table1)

### Writting data into a .csv file

In [4]:
csv_file = open('toronto.csv', 'wb')
csv_file.write(bytes(table1,encoding="ascii",errors="ignore"))

8708

### Converting it into dataframe

In [5]:
df = pd.read_csv('toronto.csv', header = None)
df.columns = ['Postalcode', 'Borough', 'Neighbourhood']
df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Droping row where borough is "Not assigned"

In [6]:
# Only processing the cells that have an assigned borough. 
# Ignoring the cells with a borough that is Not assigned.

indexNum = df[df['Borough'] == 'Not assigned'].index
df.drop(indexNum, inplace = True)
df.head(10)

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
9,M9A,Downtown Toronto,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


### Dealing the 'Not assigned' neighborhood

In [7]:
# If a cell has a borough but a Not assigned neighborhood
# then the neighborhood will be the same as the borough

df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']
df.head()

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


### Combining rows with the same postal code area

In [8]:
df_group = df.groupby(['Postalcode', 'Borough'], sort = False).agg( ','.join)
df_new = df_group.reset_index()
df_new.head()

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


### Use the .shape method to print the number of rows of your dataframe

In [9]:
df_new.shape

(103, 3)